In [54]:
from pathlib import Path 
import pandas as pd
from rgispy.routines.mkgrid import cellattr_to_grid
from rgispy.core import RgisPoint, Rgis, RgisCalculate, RgisNetwork, RgisTable, RgisGrid
import os

In [2]:
RGISARCHIVE = Path(os.environ['RGISARCHIVE3'])
NETWORK = RGISARCHIVE.joinpath('CONUS/River-Network/HydroSTN30/15min/Static/CONUS_River-Network_HydroSTN30_15min_Static.gdbn.gz')

In [3]:
#rgispy needs a location to store temporary files
os.environ['SCRATCH'] = '/scratch/danielv'

In [99]:
tmp = Path.cwd().joinpath('tmp_output')
if not tmp.exists:
    tmp.mkdir()

## Process Tutorial

Goal: Convert an attribute of geolocated points to an RGIS grid coverage (gdbc). 

In this case, we want to convert the attribute `DummyValue` to an input layer for RGIS. 

Lets first go over the base process using stock RGIS commands.

In [65]:
dams = pd.read_csv('dams_15min.csv')
dams

,Unnamed: 0,ID,grand_id,XCoord,YCoord,DummyValue
0,0,1,40,-122.370,48.870,100
1,1,2,41,-121.125,48.625,101
2,2,3,42,-120.370,48.120,102
3,3,4,43,-121.375,47.875,103
4,4,5,44,-121.375,47.875,104
5,5,6,47,-121.620,47.870,105
6,6,7,48,-121.875,47.875,107
7,7,8,49,-120.120,48.620,108
8,8,9,50,-121.625,47.625,109


Import csv as point coverage and snap to cell centers. `pntSTNCoord` will snap the point locations to the center coordinates of the network cells they fall in. If your points already snapped to the network, you don't need to do this. 

In [100]:
gdbp = RgisPoint.from_df(dams, xcol='XCoord', ycol='YCoord')
gdbp.to_file('tmp_output/dams_15min.gdbp', replace_path=True)
gdbp.pnt_stn_coord(NETWORK, )
gdbp.gdf()

,ID,Name,SymbolFLD,Unnamed: 3,Unnamed: 0,ID.1,grand_id,XCoord,YCoord,DummyValue,geometry
0,1,Record1,Default Symbol,0,0,1,40,-122.375,48.875,100,POINT (-122.37500 48.87500)
1,2,Record2,Default Symbol,1,1,2,41,-121.125,48.625,101,POINT (-121.12500 48.62500)
2,3,Record3,Default Symbol,2,2,3,42,-120.375,48.125,102,POINT (-120.37500 48.12500)
3,4,Record4,Default Symbol,3,3,4,43,-121.375,47.875,103,POINT (-121.37500 47.87500)
4,5,Record5,Default Symbol,4,4,5,44,-121.375,47.875,104,POINT (-121.37500 47.87500)
5,6,Record6,Default Symbol,5,5,6,47,-121.625,47.875,105,POINT (-121.62500 47.87500)
6,7,Record7,Default Symbol,6,6,7,48,-121.875,47.875,107,POINT (-121.87500 47.87500)
7,8,Record8,Default Symbol,7,7,8,49,-120.125,48.625,108,POINT (-120.12500 48.62500)
8,9,Record9,Default Symbol,8,8,9,50,-121.625,47.625,109,POINT (-121.62500 47.62500)


To create an RGIS grid coverage from a tabular input, we need the cell id that each point falls within. `pntSTNChar` will add the network attributes to our table, including the cell ids.

In [101]:
gdbp.pnt_stn_char(NETWORK, suffix='_15min')
gdf = gdbp.gdf()
gdf.columns

Index(['ID', 'Name', 'SymbolFLD', 'Unnamed: 3', 'Unnamed: 0', 'ID.1',
       'grand_id', 'XCoord', 'YCoord', 'DummyValue', 'CellID_15min',
       'BasinID_15min', 'BasinName_15min', 'Order_15min', 'Color_15min',
       'NumberOfCells_15min', 'STNMainstemLength_15min',
       'STNCatchmentArea_15min', 'STNInterStationArea_15min',
       'NextStation_15min', 'geometry'],
      dtype='object')

In [102]:
gdf[['ID', 'grand_id', 'CellID_15min', 'DummyValue', 'geometry']]

,ID,grand_id,CellID_15min,DummyValue,geometry
0,1,40,10970,100,POINT (-122.37500 48.87500)
1,2,41,10960,101,POINT (-121.12500 48.62500)
2,3,42,7440,102,POINT (-120.37500 48.12500)
3,4,43,10947,103,POINT (-121.37500 47.87500)
4,5,44,10947,104,POINT (-121.37500 47.87500)
5,6,47,10870,105,POINT (-121.62500 47.87500)
6,7,48,10829,107,POINT (-121.87500 47.87500)
7,8,49,7484,108,POINT (-120.12500 48.62500)
8,9,50,10943,109,POINT (-121.62500 47.62500)


Conceptually, we can now join this table `DBCells` table of the river network using the cell id a join key. The `DBCells` table contains an id and attributes for every cell in the river network. 

In [103]:
net = RgisNetwork(NETWORK)
net.db_cells().df()

,ID,Name,ToCell,FromCell,Order,BasinID,BasinCells,Travel,CellArea,CellLength,SubbasinArea,SubbasinLength
0,1,GHAASCell:1,2,49,6,1,5696,0,671.798645,36.854435,3.324476e+06,4405.626465
1,2,GHAASCell:2,2,209,6,1,5693,1,670.125366,36.815098,3.322460e+06,4368.771973
2,3,GHAASCell:3,1,48,6,1,5689,2,670.125366,24.105507,3.319783e+06,4331.957031
3,4,GHAASCell:4,2,17,6,1,5687,3,668.439331,36.775517,3.318443e+06,4307.851562
4,5,GHAASCell:5,1,52,6,1,5685,4,668.439331,24.044857,3.317106e+06,4271.076172
...,...,...,...,...,...,...,...,...,...,...,...,...
15562,15563,GHAASCell:15563,128,0,1,275,1,0,508.288483,33.248363,5.082885e+02,33.248363
15563,15564,GHAASCell:15564,128,0,1,276,1,0,508.288483,33.248363,5.082885e+02,33.248363
15564,15565,GHAASCell:15565,64,0,1,277,1,0,508.288483,27.799698,5.082885e+02,27.799698
15565,15566,GHAASCell:15566,64,0,1,278,1,0,508.288483,27.799698,5.082885e+02,27.799698


In [104]:
# convert dataframe to gdbt table
tbl_path = tmp.joinpath('dams_15min_prepped.gdbt')
tbl = RgisTable.from_df(gdf[['CellID_15min', 'DummyValue']])
tbl.to_file(tbl_path, replace_path=True)

# join temp gdbt on network dbcells table
network_joined_path = tmp.joinpath('network_15min_joined.gdbn')
network_joined = net.tbl_join_tables(
    tbl_path,
    out_dataset=network_joined_path,
    relate_table="DBCells",
    join_field='CellID_15min',
)

Our joined table is will contain `NA` for any values with no matching keys.

In [105]:
dbcells_join = network_joined.db_cells().df()
dbcells_join[~dbcells_join.DummyValue.isna()]

,ID,Name,ToCell,FromCell,Order,BasinID,BasinCells,Travel,CellArea,CellLength,SubbasinArea,SubbasinLength,DBItems,Unnamed: 13,CellID_15min,DummyValue
7439,7440,GHAASCell:7440,2,48,2,3,7,28,515.865051,33.449078,3598.466797,103.780029,Record3,2.0,7440,102.0
7483,7484,GHAASCell:7484,4,225,2,3,5,30,510.823730,27.799698,2546.512939,61.098270,Record8,7.0,7484,108.0
10828,10829,GHAASCell:10829,16,1,1,6,3,7,518.370972,18.646646,1555.112915,55.939941,Record7,6.0,10829,107.0
10869,10870,GHAASCell:10870,16,1,1,6,2,8,518.370972,18.646646,1036.741943,37.293293,Record6,5.0,10870,105.0
10942,10943,GHAASCell:10943,16,0,1,6,1,8,520.867004,18.736435,520.867004,18.736435,Record9,8.0,10943,109.0
10946,10947,GHAASCell:10947,16,0,1,6,1,9,518.370972,18.646646,518.370972,18.646646,Record4,3.0,10947,103.0
10959,10960,GHAASCell:10960,16,0,1,6,1,10,510.823730,18.375160,510.823730,18.375160,Record2,1.0,10960,101.0
10969,10970,GHAASCell:10970,64,0,1,6,1,7,508.288483,27.799698,508.288483,27.799698,Record1,0.0,10970,100.0


In [117]:
dummy_grd_path = tmp.joinpath('DummyValue_15min.gdbc')
dummy_grd = network_joined.netCells2Grid('DummyValue', dummy_grd_path)

We have successfully created our rgis grid! Again, note that all cells outside of our 9 cells will have an `NA` value. 

In [118]:
dummy_grd.db_items().df()

,ID,Name,Minimum,Maximum,Average,StdDev
0,1,DummyValue,100.0,109.0,104.389489,3.156416


Those `NA` values can be problematic for interpolation to different resolutions. `grdCalculate` canbe used to replace the NA values within network with 0.

The ternary expression below uses an arbitrary network attribute (BasinID) as a reference to all non-NA cells in the network. 


In [121]:
# create reference to network na vs not na ``
basinid_gdbc = tmp.joinpath('basinid_15min.gdbc')
net.netCells2Grid("BasinID", out_grid=basinid_gdbc)

# fill in na with override value if in valid network cell
expr = (
    f"{basinid_gdbc.relative_to(Path.cwd())} == nodata ? nodata"
    f" : ( {dummy_grd_path.relative_to(Path.cwd())} == nodata ? 0 : {dummy_grd_path.relative_to(Path.cwd())} )"
)

expr

'tmp_output/basinid_15min.gdbc == nodata ? nodata : ( tmp_output/DummyValue_15min.gdbc == nodata ? 0 : tmp_output/DummyValue_15min.gdbc )'

```
# pseduocode translation
if basinid == nodata:
    return nodata
elif dummy_grd == nodata:
    return 0
else:
    return dummy_grd.value
```

In [122]:
rcalc = RgisCalculate()
dummy_grd_zeros_path = tmp.joinpath('DummyValue_15min_zeros.gdbc')
rcalc.grdCalculate(expr, dummy_grd_zeros_path, extent=NETWORK)

PosixPath('/asrc/ecr/danielv/projects/rgispy/examples/layer_creation/tmp_output/DummyValue_15min_zeros.gdbc')

The minimum is now zero. The na network cells have been successfully replaced. 

In [123]:
RgisGrid(dummy_grd_zeros_path).db_items().df()

,ID,Name,Minimum,Maximum,Average,StdDev
0,1,DummyValue,0.0,109.0,0.285492,5.440201


## Convenience Function

The `rgispy.routines.mkgrid.cellattr_to_grid` function will perform the process above given a dataframe with cell ids & and a rgis network.

In [124]:
dummy_grd_2 = tmp.joinpath('DummyValue_15min_zeros_2.gdbc')
cellattr_to_grid(gdf, 'DummyValue', NETWORK, dummy_grd_2, na_override=0, cellid_field='CellID_15min')

AssertionError: CellID must be unique in dataframe

One of the cells has two associated values. The manual rgis method simply took the first value and dropped the extra without warning. `cellattr_to_grid` requires one associated value per cell id.

In [125]:
gdf[['CellID_15min', 'DummyValue']]

,CellID_15min,DummyValue
0,10970,100
1,10960,101
2,7440,102
3,10947,103
4,10947,104
5,10870,105
6,10829,107
7,7484,108
8,10943,109


In this case, we will just arbitrarily drop one of the records. 

In [126]:
gdf = gdf.drop_duplicates(subset='CellID_15min')
gdf[['CellID_15min', 'DummyValue']]

,CellID_15min,DummyValue
0,10970,100
1,10960,101
2,7440,102
3,10947,103
5,10870,105
6,10829,107
7,7484,108
8,10943,109


In [128]:
cellattr_to_grid(
    gdf,
    "DummyValue",
    NETWORK,
    dummy_grd_2.relative_to(Path.cwd()),
    na_override=0,
    cellid_field="CellID_15min",
)


PosixPath('tmp_output/DummyValue_15min_zeros_2.gdbc')

The same result is achieved!

In [129]:
RgisGrid(dummy_grd_2).db_items().df()

,ID,Name,Minimum,Maximum,Average,StdDev
0,1,DummyValue,0.0,109.0,0.285492,5.440201


## Bash Version

For completeness, here is the same process using the RGIS commands in bash

`$1` is the full path to the 15 minute river network in the below scripts.

In [130]:
%%bash -s "$NETWORK"
export PATH="$PATH:/usr/local/share/ghaas/bin:/usr/local/share/ghaas/f"

pntSTNCoord --network $1 tmp_output/dams_15min.gdbp tmp_output/dams_15min_coord.gdbp

pntSTNChar --network $1 tmp_output/dams_15min_coord.gdbp tmp_output/dams_15min_char.gdbp

tblJoinTables -a tmp_output/dams_15min_char.gdbp -e "DBCells" -j "CellID" "$1" tmp_output/network_15min_joined_manual.gdbn

netCells2Grid -f "DummyValue" tmp_output/network_15min_joined_manual.gdbn tmp_output/DummyValue_15min_manual.gdbc 

rgis2table tmp_output/DummyValue_15min_manual.gdbc

"ID"	"Name"	"Minimum"	"Maximum"	"Average"	"StdDev"
1	"DummyValue"	100.000000	109.000000	104.389489	3.156416


In [132]:
%%bash -s "$NETWORK"
export PATH="$PATH:/usr/local/share/ghaas/bin:/usr/local/share/ghaas/f"
netCells2Grid -f "BasinID" $1 tmp_output/basinid_15min_manual.gdbc 
grdCalculate -x $1 -c "tmp_output/basinid_15min_manual.gdbc == nodata ? nodata : ( tmp_output/DummyValue_15min_manual.gdbc == nodata ? 0 : tmp_output/DummyValue_15min_manual.gdbc )" \
    > tmp_output/DummyValue_15min_manaul_zeros.gdbc
rgis2table tmp_output/DummyValue_15min_manaul_zeros.gdbc

"ID"	"Name"	"Minimum"	"Maximum"	"Average"	"StdDev"
1	"DummyValue"	0.000000	109.000000	0.285492	5.440201


In [134]:
%%bash
# cleanup
rm -r tmp_output